In [1]:
import numpy as np

In [2]:
from skimage.transform import resize

In [3]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Define relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [5]:
from tqdm.notebook import tqdm

In [9]:
def get_dataset(dataset):
  if dataset == 'mnist':
    train_dataset = torchvision.datasets.MNIST(root = './data',
                                              train = True,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                              download = True)


    test_dataset = torchvision.datasets.MNIST(root = './data',
                                              train = False,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                              download=True)
  elif dataset == 'usps':
    train_dataset = torchvision.datasets.USPS(root = './data',
                                              train = True,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                              download = True)


    test_dataset = torchvision.datasets.USPS(root = './data',
                                              train = False,
                                              transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                              download=True)
  elif dataset == 'svhn':
    train_dataset = torchvision.datasets.SVHN(root = './data',
                                              split = 'train',
                                              transform = transforms.Compose([
                                                      transforms.Grayscale(),
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                              download = True)


    test_dataset = torchvision.datasets.SVHN(root = './data',
                                              split = 'test',
                                              transform = transforms.Compose([
                                                      transforms.Grayscale(),
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                              download=True)
    


  train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = batch_size,
                                            shuffle = True)


  test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                            batch_size = batch_size,
                                            shuffle = True)

  return train_loader, test_loader

In [10]:
def train_and_evaluate(dataset):
  model = LeNet5(num_classes).to(device)
  cost = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  train_loader, test_loader = get_dataset(dataset)
  total_step = len(train_loader)

  for epoch in tqdm(range(num_epochs)):
      for i, (images, labels) in enumerate(train_loader):  
          images = images.to(device)
          labels = labels.to(device)
          
          outputs = model(images)
          loss = cost(outputs, labels)
            
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
  
  print(f"LeNet-5 trained on {dataset}")
  for dataset_name in ['mnist', 'usps', 'svhn']:
    with torch.no_grad():
      correct = 0
      total = 0
      _, test_loader = get_dataset(dataset_name)
      for images, labels in test_loader:
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

      print(f'Accuracy {dataset_name}: {100 * correct / total:.2f}%')

In [11]:
train_and_evaluate('mnist')

  0%|          | 0/10 [00:00<?, ?it/s]

LeNet-5 trained on mnist
Accuracy mnist: 99.17%
Accuracy usps: 78.08%


  0%|          | 0/182040794 [00:00<?, ?it/s]

  0%|          | 0/64275384 [00:00<?, ?it/s]

Accuracy svhn: 31.50%


In [12]:
train_and_evaluate('usps')

  0%|          | 0/10 [00:00<?, ?it/s]

LeNet-5 trained on usps
Accuracy mnist: 57.10%
Accuracy usps: 95.42%
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Accuracy svhn: 26.94%


In [13]:
train_and_evaluate('svhn')

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


  0%|          | 0/10 [00:00<?, ?it/s]

LeNet-5 trained on svhn
Accuracy mnist: 61.92%
Accuracy usps: 64.28%
Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
Accuracy svhn: 89.52%
